In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=1800727326928) was called 1 times


2021-01-04 10:32:21,474 - NumExpr defaulting to 8 threads.


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=1800727329520) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=1800727330672) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=1800727328512) was called 1 times
Running: Function "load_module_lookup" (id=1800727330240) was called 1 times
Running: Function "load_ID_lookup" (id=1800727328944) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [9]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av)

Running: Function "load_PTT_lookup" (id=1800727328656) was called 1 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1800727328368) was called 1 times
Running: Function "load_PTT_lookup" (id=1800727328656) was called 2 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1800727328368) was called 2 times
Running: Function "load_PTT_lookup" (id=1800727328656) was called 3 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1800727328368) was called 3 times
Running: Function "load_PTT_lookup" (id=1800727328656) was called 4 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1800727328368) was called 4 times
Running: Function "load_PTT_lookup" (id=1800727328656) was called 5 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=1800727328368) was called 5 times
Running: Function "load_PTT_lookup" (id=1800727328656) was called 6 times
Previous Hours Weighted
Running

In [10]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=1-y

In [11]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

model,pred,coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

n=30

md.run_OLS(X_train,y_train,X_test,y_test, n)

#Problem with this function!



C:\Users\Jamie.williams\Anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


292
                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.958
Model:                            OLS   Adj. R-squared (uncentered):              0.957
Method:                 Least Squares   F-statistic:                              929.6
Date:                Mon, 04 Jan 2021   Prob (F-statistic):                        0.00
Time:                        10:35:07   Log-Likelihood:                          6908.2
No. Observations:               11899   AIC:                                 -1.324e+04
Df Residuals:                   11611   BIC:                                 -1.111e+04
Df Model:                         288                                                  
Covariance Type:            nonrobust                                                  
                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------

TypeError: not all arguments converted during string formatting

### Add LM - metrics

In [ ]:
X_train_rd = X_train
Remove = 30

def find_features(X_train, y_train, X):
    max_p = 1
    while max_p > 0.1:
        model = sm.OLS(y_train, X_train)
        results = model.fit()
        top_X = results.pvalues.sort_values(ascending=False).head(X)
        max_p = top_X.tail(1).values[0]
        rm_col = list(results.pvalues.sort_values(ascending=False).head(X).index)
        X_train = X_train.drop(rm_col, axis=1)
    return X_train.columns

In [15]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

C0904STA206    7
Name: Asset Code, dtype: int64

In [ ]:
fa[fa['Fault ID']==7647]

In [ ]:
fa[fa['Pick Station']!=False]['Asset Code'].value_counts()

In [ ]:
pick_stations = ['PTT011','PTT012','PTT021','PTT022','PTT031','PTT032','PTT041','PTT042','PTT051','PTT052','PTT071','PTT072','PTT081','PTT082','PTT091','PTT092','PTT101','PTT102','PTT111','PTT112','PTT121','PTT122','PTT131','PTT132','PTT141','PTT142','PTT151','PTT152','PTT171','PTT172','PTT181','PTT182','PTT191','PTT192','PTT201','PTT202']

In [ ]:
fa_floor[fa_floor['Duration']==0]

In [12]:
unmapped

,Asset,Occurrence
0,C08,26
1,C06,9
2,C0904STA206,7
3,C09,4
4,C13,4
5,C14,4
6,C05,3
7,C10,3
8,C11,3
9,C12,1


In [14]:
fa[fa['Asset Code'] == 'C08']

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
168514,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:32,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168515,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:28,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168513,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:34,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168512,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:36,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168511,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:54,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168525,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:57,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168527,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:42,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168526,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:56,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168528,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:40,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168517,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:22,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault


In [23]:
np.exp(X/100)-1

,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,C0501STA201,C0501STA217,C0501STA219,C0501STA228,...,SCS202,SCS203,SCS204,SCS205,SCS206,SCSM20,C1404RDC146,C1404STA149,C14PTT202,TOTES
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.436491
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.384723
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.266596
3,0.0,0.0,0.0,0.0,0.072508,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.398406
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.390041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,29.569415,4.365556,0.000000,7.124044e+01,0.0,0.447735,0.0,0.000000,0.000000,0.636675
470,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,20.115344,16.814273,1.104419e+08,0.0,2.596640,0.0,0.000000,0.000000,0.808805
471,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,316.348329,0.000000,7.004469,6.108268e+04,0.0,0.000000,0.0,89.921819,1.974274,0.710296
472,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,234.097424,5.233887e+00,0.0,0.000000,0.0,0.000000,0.051271,0.688206
